## What this notebook is
This notebook demonstrates how I trained Gemma-2 9b to obtain LB: 0.941. The inference code can be found [here](https://www.kaggle.com/code/emiz6413/inference-gemma-2-9b-4-bit-qlora).
I used 4-bit quantized [Gemma 2 9b Instruct](https://huggingface.co/unsloth/gemma-2-9b-it-bnb-4bit) uploaded by unsloth team as a base-model and added LoRA adapters and trained for 1 epoch.

## Result

I used `id % 5 == 0` as an evaluation set and used all the rest for training.

| subset | log loss |
| - | - |
| eval | 0.9371|
| LB | 0.941 |

## What is QLoRA fine-tuning?

In the conventional fine-tuning, weight ($\mathbf{W}$) is updated as follows:

$$
\mathbf{W} \leftarrow \mathbf{W} - \eta \frac{{\partial L}}{{\partial \mathbf{W}}} = \mathbf{W} + \Delta \mathbf{W}
$$

where $L$ is a loss at this step and $\eta$ is a learning rate.

[LoRA](https://arxiv.org/abs/2106.09685) tries to approximate the $\Delta \mathbf{W} \in \mathbb{R}^{\text{d} \times \text{k}}$ by factorizing $\Delta \mathbf{W}$ into two (much) smaller matrices, $\mathbf{B} \in \mathbb{R}^{\text{d} \times \text{r}}$ and $\mathbf{A} \in \mathbb{R}^{\text{r} \times \text{k}}$ with $r \ll \text{min}(\text{d}, \text{k})$.

$$
\Delta \mathbf{W}_{s} \approx \mathbf{B} \mathbf{A}
$$

<img src="https://storage.googleapis.com/pii_data_detection/lora_diagram.png">

During training, only $\mathbf{A}$ and $\mathbf{B}$ are updated while freezing the original weights, meaning that only a fraction (e.g. <1%) of the original weights need to be updated during training. This way, we can reduce the GPU memory usage significantly during training while achieving equivalent performance to the usual (full) fine-tuning.

[QLoRA](https://arxiv.org/abs/2305.14314) pushes the efficiency further by quantizing LLM. For example, a 8B parameter model alone would take up 32GB of VRAM in 32-bit, whereas quantized 8-bit/4-bit 8B model only need 8GB/4GB respectively. 
Note that QLoRA only quantize LLM's weights in low precision (e.g. 8-bit) while the computation of forward/backward are done in higher precision (e.g. 16-bit) and LoRA adapter's weights are also kept in higher precision.

1 epoch using A6000 took ~15h in 4-bit while 8-bit took ~24h and the difference in log loss was not significant.

## Note
It takes prohivitively long time to run full training on kaggle kernel. I recommend to use external compute resource to run the full training.
This notebook uses only 100 samples for demo purpose, but everything else is same as my setup.

In [1]:
import os

os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

## Gemma2_9b模型
    Gemma-2-9B 是一个大型语言模型，属于 Gemma 系列，具有 90 亿参数。

    模型架构
        参数数量：90 亿
        架构：基于 Transformer 架构，优化了注意力机制和参数效率。

    训练数据
        数据来源：从多种来源收集，包括互联网文本、书籍、学术文章等。
        多样性：覆盖广泛的主题，如科学、技术、文化、娱乐。

    性能特点
        语言理解：在自然语言处理任务中表现优异，如问答、翻译、文本生成。
        多语言支持：具备多语言理解能力，适用于多语言环境。

    应用场景
        内容生成：用于生成高质量的文本内容，如文章、报告等。
        对话系统：可用于构建智能聊天机器人。
        信息检索：提升搜索和信息提取能力。

    优化和改进
        高效推理：通过优化模型结构，实现更快速的推理速度。
        能耗管理：在训练和推理过程中优化了能耗。

#### Hugging face
    https://huggingface.co/models

    google:
    https://huggingface.co/google/gemma-2-9b-it
    
    unsloth:
    https://huggingface.co/unsloth/gemma-2-9b-it-bnb-4bit

In [2]:
import os
import copy
from dataclasses import dataclass

import numpy as np
import torch
from datasets import Dataset

# os: 用于与操作系统交互，如读取环境变量、文件路径操作等。
# copy: 提供复制对象的功能，尤其是复杂的对象如列表、字典等。
# dataclass: 用于定义具有初始化、比较等功能的类，简化类定义。
# torch: 是PyTorch框架的核心，用于构建和训练神经网络。
# Dataset: 来自datasets库，用于处理和加载数据集。


from transformers import (
    BitsAndBytesConfig,
    Gemma2ForSequenceClassification,
    GemmaTokenizerFast,
    Gemma2Config,
    PreTrainedTokenizerBase, 
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)

# BitsAndBytesConfig: 可能是用于配置模型的类，特别是位和字节级的操作。
# Gemma2ForSequenceClassification: 是一个预训练模型，用于序列分类任务，基于Gemma模型架构。
# GemmaTokenizerFast: 用于Gemma模型的快速分词器。
# Gemma2Config: Gemma模型的配置。
# PreTrainedTokenizerBase: 所有预训练分词器的基类。
# EvalPrediction: 用于处理评估阶段的预测结果。
# Trainer: 用于训练和评估Transformer模型。
# TrainingArguments: 用于配置训练过程的参数。
# DataCollatorWithPadding: 在数据批处理时自动添加填充，保证批中的所有样本具有相同长度。


from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

# 矩阵的秩 低秩优化（近似）
# LoraConfig: 用于配置Lora (Low-Rank Adaptation) 相关参数，可能是模型微调的一种方法。
# get_peft_model: 用于获取符合PEFT（可能是一种模型优化或改进技术）标准的模型。
# prepare_model_for_kbit_training: 准备模型以支持k位训练，通常涉及模型量化或其他形式的压缩。
# TaskType: 枚举或类，用于定义任务类型。

from sklearn.metrics import log_loss, accuracy_score

### Peft

    在大型语言模型（LLM）中，PEFT 是指「参数高效微调」（Parameter-Efficient Fine-Tuning）。这是对大型模型进行微调的一种方法，旨在减少所需的计算资源和参数调整量。

    PEFT的特点:
    参数效率：只调整模型的一小部分参数，而不是全部参数。
    资源节省：降低显存和计算需求，适合在资源有限的情况下使用。
    快速适应：能够快速适应新任务或领域，而无需重新训练整个模型。

    常用技术:
    Adapter Layers：在模型的特定层中插入适配层，仅微调这些层。
    LoRA (Low-Rank Adaptation)：通过低秩矩阵分解来实现高效调整。
    Prefix Tuning：调整模型的输入前缀部分而非主体结构。

    应用场景
    多任务学习：在多个任务中共享一个基础模型，仅微调特定任务所需部分。
    跨领域适应：快速适应不同的行业或领域需求。

In [3]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

### Configurations

In [4]:
@dataclass
class Config:
    # 表示输出目录，默认为 "output"。用于保存训练过程中生成的文件，如模型权重、日志等。
    output_dir: str = "output"

    # 指定预训练模型的路径或标识符，默认为 "unsloth/gemma-2-9b-it-bnb-4bit"。这表明使用的是一个量化到4位的Gemma-2模型。
    checkpoint: str = "unsloth/gemma-2-9b-it-bnb-4bit"  # 4-bit quantized gemma-2-9b-instruct
    
    # 指定处理文本的最大长度，默认为 1024。
    max_length: int = 1024
    
    # 交叉验证的分割数量
    n_splits: int = 100
    fold_idx: int = 0
    
    # 优化器类型，默认为 "adamw_8bit"。这表明使用8位优化版本的AdamW优化器。
    optim_type: str = "adamw_8bit"
   
    # 每个设备上的训练批次大小，默认为 2。
    per_device_train_batch_size: int = 2
    
    # 梯度累积步骤数，默认为 16。这实际上决定了全局批次大小为 2 * 16 = 32。
    gradient_accumulation_steps: int = 16  # global batch size is 8 
    
    # 每个设备上的评估批次大小，默认为 2。
    per_device_eval_batch_size: int = 2
    
    # 训练的轮数，默认为 1。
    n_epochs: int = 1
    
    # 冻结的隐藏层数量
    freeze_layers: int = 0  # there're 42 layers in total, we don't add adapters to the first 16 layers
    
    # 学习率
    lr: float = 0.00017
    
    # 预热步骤数，默认为 20。这是训练开始时学习率逐渐提升到设定值的步骤数。
    warmup_steps: int = 20
    
    # LoRA（Low-Rank Adaptation）的秩，默认为 64。这是一种模型适配技术，通常用于参数效率的模型调整。
    lora_r: int = 64
    
    # LoRA的缩放因子，默认为 32。
    lora_alpha: float = 32
    
    # LoRA层中使用的dropout比率，默认为 0.05。
    lora_dropout: float = 0.05
    lora_bias: str = "none"
    
config = Config()

#### Training Arguments

In [5]:
training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    report_to="none",
    num_train_epochs=config.n_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="steps",
    save_steps=500,
    optim=config.optim_type,
    fp16=True,
    learning_rate=config.lr,
    warmup_steps=config.warmup_steps,
)

#### LoRA config

#### 在 LoraConfig 中指定的 target_modules 参数 ["k_proj", "o_proj", "q_proj", "v_proj", "gate_proj"] 涉及了几个关键的模块，通常这些模块是在自注意力机制中的组成部分。

#### 自注意力机制的组成
    在自注意力机制中，输入序列通过一系列变换得到不同的表示，这些表示用于计算注意力得分和最终的输出。具体的模块包括：
    k_proj (Key Projection)：键投影，用于生成键向量。在注意力机制中，键（Key）用于与查询（Query）进行匹配，以计算注意力得分。
    q_proj (Query Projection)：查询投影，用于生成查询向量。查询向量与所有键向量的匹配程度决定了每个元素对输出的影响程度。
    v_proj (Value Projection)：值投影，用于生成值向量。值（Value）是在计算得到的注意力得分基础上，对输入的加权求和，即实际上被聚焦的信息。
    o_proj (Output Projection)：输出投影，用于将注意力机制的输出转换为下一层或最终输出所需的格式。
    gate_proj (Gate Projection)：门控投影，这是一种较少见的模块，可能用于实现一种门控机制，类似于在LSTM或GRU中的门控。门控可以控制信息的流动，例如通过一个sigmoid函数来决定信息的保留与丢弃程度。

In [6]:
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    # only target self-attention
    target_modules=["k_proj", "o_proj", "q_proj", "v_proj", "gate_proj"],
    layers_to_transform=[i for i in range(42) if i >= config.freeze_layers],
    lora_dropout=config.lora_dropout,
    bias=config.lora_bias,
    task_type=TaskType.SEQ_CLS,
)

### Instantiate the tokenizer & model

In [7]:
tokenizer = GemmaTokenizerFast.from_pretrained(config.checkpoint)
tokenizer.add_eos_token = True  # We'll add <eos> at the end
tokenizer.padding_side = "right"

In [8]:
model = Gemma2ForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=3,
    torch_dtype=torch.float16,
    device_map="auto",
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at unsloth/gemma-2-9b-it-bnb-4bit and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): Gemma2ForSequenceClassification(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 3584, padding_idx=0)
        (layers): ModuleList(
          (0-41): 42 x Gemma2DecoderLayer(
            (self_attn): Gemma2SdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3584, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): Modul

In [9]:
model.print_trainable_parameters()

trainable params: 119,745,024 || all params: 9,361,461,760 || trainable%: 1.2791


### Instantiate the dataset

In [10]:
import pandas as pd

train_1 = pd.read_csv('train.csv', index_col = 0)
train_2 = pd.read_csv('lmsys-33k-deduplicated.csv', index_col=0)

train = pd.concat([train_1, train_2], axis=0, ignore_index=True)
# ds = ds.select(torch.arange(100))  # We only use the first 100 data for demo purpose

In [11]:
ds = Dataset.from_pandas(train)

In [12]:
class CustomTokenizer:

    # __init__ 方法
    
    # 参数：
    #   tokenizer: 一个 PreTrainedTokenizerBase 类型的对象，通常来源于诸如Hugging Face的Transformers库。这种分词器包含了预训练模型的词汇表和分词规则。
    #   max_length: 一个整数，定义了分词后的最大长度，超过这个长度的输入将被截断。
    
    # 属性：
    # self.tokenizer: 存储传入的分词器实例。
    # self.max_length: 存储最大长度限制。
    def __init__(
        self, 
        tokenizer: PreTrainedTokenizerBase, 
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    
    # __call__ 方法
    #   这个方法使得 CustomTokenizer 实例可以像函数一样被调用，并处理包含多个文本输入的批量数据。

    #   输入：一个字典 batch，它包含键如 prompt, response_a, response_b, winner_model_a, 和 winner_model_b。
    
    #   处理流程：
    #       文本预处理：通过 process_text 方法处理 prompt, response_a, 和 response_b 中的每个文本。这个方法会在每个文本前添加指定的标签（如 <prompt>:），用于明确文本的角色。
    #       文本合并：将处理后的 prompt, response_a, 和 response_b 合并成单一的文本串，以便进行分词。
    #       分词：使用 self.tokenizer 对合并后的文本进行分词，应用最大长度限制和截断。
    
    #   标签处理：根据 winner_model_a 和 winner_model_b 的布尔值来确定标签（0表示 a 胜，1表示 b 胜，2表示平局）。
    def __call__(self, batch: dict) -> dict:
        prompt = ["<prompt>: " + self.process_text(t) for t in batch["prompt"]]
        response_a = ["\n\n<response_a>: " + self.process_text(t) for t in batch["response_a"]]
        response_b = ["\n\n<response_b>: " + self.process_text(t) for t in batch["response_b"]]
        texts = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)
        labels=[]
        for a_win, b_win in zip(batch["winner_model_a"], batch["winner_model_b"]):
            if a_win:
                label = 0
            elif b_win:
                label = 1
            else:
                label = 2
            labels.append(label)
        return {**tokenized, "labels": labels}
    
    # process_text 静态方法
    # 输入：一个字符串 text。
    # 处理：使用 eval 函数解析 text（假设 text 是一个Python表达式形式的字符串），并将其中的 null 替换为空字符串。然后将解析后的列表转换为单个字符串。
    # 注意：使用 eval 可能会带来安全风险，因为它允许执行任意代码。在实际应用中，如果输入来源不可控，应避免使用 eval 或使用安全的替代方法。
    @staticmethod
    def process_text(text: str) -> str:
        return " ".join(eval(text, {"null": ""}))

In [13]:
encode = CustomTokenizer(tokenizer, max_length=config.max_length)
ds = ds.map(encode, batched=True)

# .map() 方法是 datasets 库中处理数据集的常用方法，它允许用户应用一个函数到数据集中的每个元素。
# 在这个情况下，函数是 encode 实例，它根据定义的逻辑处理每个批次的数据。

# batched=True 参数指示 .map() 方法将数据以批次的形式传递给 encode 函数，而不是单个元素。
# 这样做的好处是可以提高处理速度，因为分词器通常对批量数据的处理更加高效。

Map:   0%|          | 0/78664 [00:00<?, ? examples/s]

<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequen

### Compute metrics

We'll compute the log-loss used in LB and accuracy as a auxiliary metric.

In [14]:
def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions
    labels = eval_preds.label_ids
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()
    loss = log_loss(y_true=labels, y_pred=probs)
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))
    return {"acc": acc, "log_loss": loss}

### Split

Here, train and eval is splitted according to their `id % 100`

In [15]:
folds = [
    (
        [i for i in range(len(ds)) if i % config.n_splits != fold_idx],
        [i for i in range(len(ds)) if i % config.n_splits == fold_idx]
    ) 
    for fold_idx in range(config.n_splits)
]

In [ ]:
train_idx, eval_idx = folds[config.fold_idx]

trainer = Trainer(
    args=training_args, 
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds.select(train_idx),
    eval_dataset=ds.select(eval_idx),
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)
trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/root/miniconda3/lib/python3.12/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
